<a href="https://colab.research.google.com/github/Bobbsicle27/VandanaK/blob/main/Clinician_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fpdf

# Clinician report code

In [ ]:
# Import necessary libraries
import pandas as pd
from fpdf import FPDF
import matplotlib.pyplot as plt
import numpy as np
import os
import zipfile
from google.colab import files

# Function to upload and read Excel files with headers in row 2 (header=1)
def upload_and_read_excel(prompt):
    print(f"Upload the {prompt} file:")
    uploaded = files.upload()  # Prompt user to upload a file
    for fn in uploaded.keys():  # Load the uploaded file into a pandas DataFrame
        return pd.read_excel(fn, header=1)  # Read the headers from the second row

# Upload necessary files
cashbook_df = upload_and_read_excel("cashbook")
average_daily_yield_df = upload_and_read_excel("average daily yield")
appointments_df = upload_and_read_excel("appointments")
chair_occupancy_df = upload_and_read_excel("chair occupancy")
rebooking_df = upload_and_read_excel("rebooking rates")
treatment_counts_df = upload_and_read_excel("treatment counts")  # File with treatment type and count per clinician
historical_data_df = upload_and_read_excel("historical data")  # Upload historical data file
treatment_plan_analysis_df = upload_and_read_excel("future treatment plan analysis")  # Upload future treatment plan file

# Helper function to clean and normalize clinician names
def clean_clinician_name(name):
    return ' '.join(name.split()).strip().lower()

# Normalize columns to avoid mismatches during merges
appointments_df['Clinician'] = appointments_df['Clinician'].apply(clean_clinician_name)
cashbook_df['User/Shop'] = cashbook_df['User/Shop'].apply(clean_clinician_name)
average_daily_yield_df['User Name'] = average_daily_yield_df['User Name'].apply(clean_clinician_name)
chair_occupancy_df['Clinician'] = chair_occupancy_df['Clinician'].apply(clean_clinician_name)
rebooking_df['Clinician'] = rebooking_df['Clinician'].apply(clean_clinician_name)
treatment_counts_df['Clinician'] = treatment_counts_df['Clinician'].apply(clean_clinician_name)
historical_data_df['Clinician'] = historical_data_df['Clinician'].apply(clean_clinician_name)
treatment_plan_analysis_df['clinician'] = treatment_plan_analysis_df['clinician'].apply(clean_clinician_name)

# Convert ADY to numeric, coercing any non-numeric values to NaN and forward fill
historical_data_df['ADY'] = pd.to_numeric(historical_data_df['ADY'], errors='coerce').ffill()
historical_data_df['Rebooking Percentage'] = pd.to_numeric(historical_data_df['Rebooking Percentage'], errors='coerce').ffill()

# Clean appointments data and normalize clinician names
appointments_clean = appointments_df[['Forename', 'Surname', 'Appointment Type', 'Appointment Status', 'Clinician']].copy()
appointments_clean = appointments_clean[appointments_clean['Appointment Status'] == 'Completed']
appointments_clean['Full Name'] = appointments_clean['Forename'].str.strip() + ' ' + appointments_clean['Surname'].str.strip()

# Count unique patients per clinician
unique_patients_per_clinician = appointments_clean.groupby('Clinician')['Full Name'].nunique().reset_index()
unique_patients_per_clinician.columns = ['Clinician', 'Unique Patients Seen']

# Clean and extract relevant columns from both cashbook and ADY datasets
cashbook_df_clean = cashbook_df[['User/Shop', 'Amount']].copy()
ady_df_clean = average_daily_yield_df[['User Name', 'Days Worked']].copy()

# Sum up turnover per clinician
cashbook_df_clean['Amount'] = pd.to_numeric(cashbook_df_clean['Amount'], errors='coerce')
cashbook_df_clean.dropna(subset=['Amount'], inplace=True)
turnover_per_clinician = cashbook_df_clean.groupby('User/Shop')['Amount'].sum().reset_index()

# Merge turnover with days worked
final_df = pd.merge(turnover_per_clinician, ady_df_clean, left_on='User/Shop', right_on='User Name', how='inner')
final_df['ADY'] = final_df['Amount'] / final_df['Days Worked']

# Merge unique patient counts and chair occupancy data
final_df_with_patients = pd.merge(final_df, unique_patients_per_clinician, left_on='User/Shop', right_on='Clinician', how='left')
final_df_with_patients = pd.merge(final_df_with_patients, chair_occupancy_df[['Clinician', 'Chair Occupancy (%)']], on='Clinician', how='left')
final_df_with_patients['Chair Occupancy (%)'] = final_df_with_patients['Chair Occupancy (%)'].fillna(0)

# Merge with rebooking percentages
final_df_with_patients = pd.merge(final_df_with_patients, rebooking_df[['Clinician', 'Rebooking Percentage (%)']], on='Clinician', how='left')

# Merge the pre-processed treatment plan analysis
final_df_with_patients = pd.merge(
    final_df_with_patients,
    treatment_plan_analysis_df[['clinician', 'percentage_with_future_treatment_plans']],
    left_on='User/Shop',
    right_on='clinician',
    how='left'
)

# Manually input Group Average ADY
group_avg_ady = float(input("Enter Group Average ADY for the month: "))

# Function to generate line graph for the given clinician and metric (ADY, Chair Occupancy, or Rebooking Percentage)
def generate_line_graph(clinician_name, df, metric_name):
    clinician_data = df[df['Clinician'] == clinician_name]
    if clinician_data[metric_name].isna().all():
        print(f"No {metric_name} data available for {clinician_name}")
        return None

    clinician_data.loc[:, 'Month/Year'] = pd.to_datetime(clinician_data['Month/Year'], format='%Y-%m-%d')
    clinician_data = clinician_data.sort_values(by='Month/Year')

    plt.figure(figsize=(6, 3))  # Smaller graph size
    plt.plot(clinician_data['Month/Year'], clinician_data[metric_name], marker='o', linestyle='-', color='b')
    plt.title(f"{clinician_name.title()} - {metric_name} Month-on-Month Trends")
    plt.xlabel('Month')
    plt.ylabel(metric_name)
    plt.xticks(rotation=45)
    plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter("%m-%Y"))
    plt.grid(True)

    graph_filename = f"{clinician_name}_trend_{metric_name}.png"
    plt.savefig(graph_filename, bbox_inches='tight')
    plt.close()
    return graph_filename

# Function to generate a pie chart for treatments with horizontal legend and top 10 treatments
def generate_pie_chart_with_arrows(clinician_name, treatment_types):
    plt.figure(figsize=(8, 4))  # Smaller chart size

    # Sort treatments by count and select top 10 treatments
    treatment_types = sorted(treatment_types, key=lambda x: x[1], reverse=True)[:10]
    labels = [t[0] for t in treatment_types]
    sizes = [t[1] for t in treatment_types]
    other_size = sum([t[1] for t in treatment_types[10:]])

    # Add "Other" category if needed
    if other_size > 0:
        labels.append("Other")
        sizes.append(other_size)

    # Generate pie chart with autopct to get percentages
    wedges, texts, autotexts = plt.pie(sizes, startangle=90, wedgeprops=dict(width=0.3), autopct=lambda pct: f"{pct:.1f}%", pctdistance=0.85)

    # Horizontal legend with categories and percentages
    plt.legend(wedges, [f"{label}: {size} ({size / sum(sizes) * 100:.1f}%)" for label, size in zip(labels, sizes)],
               title="Top 10 Treatments", loc="center left", bbox_to_anchor=(1, 0.5))
    plt.title(f'{clinician_name.title()} - Breakdown of Treatments Done', pad=20)
    plt.axis('equal')

    # Save chart as an image
    chart_filename = f"{clinician_name}_treatment_pie_chart.png"
    plt.savefig(chart_filename, bbox_inches='tight')
    plt.close()

    return chart_filename

# Function to generate the PDF report
def generate_pdf(clinician_name, month_year, turnover, ady, group_avg_ady, days_worked, unique_patients, treatment_types, chair_occupancy, rebooking_percentage, future_plan_percentage, ady_trend_graph, chair_occupancy_trend_graph, rebooking_trend_graph):
    pdf = FPDF()
    pdf.add_page()

    # Title: Clinician Name and Date
    pdf.set_font('Arial', 'B', 14)
    pdf.cell(190, 10, f"{clinician_name.title()}", ln=True, align='C')
    pdf.set_font('Arial', 'I', 10)
    pdf.cell(190, 6, f"{month_year}", ln=True, align='C')

    pdf.ln(5)  # Space before the metrics section

    # Clinician metrics section in specified order
    pdf.set_font('Arial', '', 10)
    pdf.cell(190, 5, f"- Turnover by clinician = {turnover:.2f}", ln=True)
    pdf.cell(190, 5, f"- Days worked in the month = {days_worked}", ln=True)
    pdf.cell(190, 5, f"- ADY in the month = {ady:.2f}", ln=True)
    pdf.set_text_color(255, 0, 0)
    pdf.cell(190, 5, f"- Group Average ADY = {group_avg_ady:.2f}", ln=True)
    pdf.set_text_color(0, 0, 0)

    # ADY trend graph
    if ady_trend_graph:
        pdf.ln(5)
        pdf.cell(190, 5, "- ADY Month-on-Month Trends", ln=True)
        pdf.image(ady_trend_graph, x=25, y=pdf.get_y() + 5, w=140, h=60)  # Adjusted width and height

    # Chair occupancy
    pdf.ln(65)  # Space after the ADY graph
    pdf.cell(190, 5, f"- Chair occupancy (%) = {chair_occupancy:.2f}", ln=True)

    # Chair occupancy trend graph
    if chair_occupancy_trend_graph:
        pdf.ln(5)
        pdf.cell(190, 5, "- Chair Occupancy Month-on-Month Trends", ln=True)
        pdf.image(chair_occupancy_trend_graph, x=25, y=pdf.get_y() + 5, w=140, h=60)

    # Total unique patients seen and future treatment plans
    pdf.ln(65)  # Space after the Chair Occupancy graph
    pdf.cell(190, 5, f"- Total unique patients seen = {unique_patients}", ln=True)
    pdf.cell(190, 5, f"- Percentage of patients with a future treatment plan = {future_plan_percentage:.2f}%", ln=True)

    # Rebooking percentage
    pdf.cell(190, 5, f"- Rebooking Percentage (%) = {rebooking_percentage:.2f}", ln=True)

    # Rebooking percentage trend graph
    if rebooking_trend_graph:
        pdf.ln(5)
        pdf.cell(190, 5, "- Rebooking Percentage Month-on-Month Trends", ln=True)
        pdf.image(rebooking_trend_graph, x=25, y=pdf.get_y() + 5, w=140, h=60)

    # Move to a new page for treatments
    pdf.add_page()

    # Treatments table
    pdf.set_font('Arial', 'B', 10)
    pdf.cell(190, 6, "Treatments done in the month", ln=True, align='C')
    pdf.set_font('Arial', '', 8)
    pdf.ln(5)

    pdf.cell(90, 6, "Treatment Type", border=1, align='C')
    pdf.cell(30, 6, "Treatment Count", border=1, align='C')
    pdf.ln()
    for treatment, count in treatment_types:
        pdf.cell(90, 6, treatment, border=1, align='C')
        pdf.cell(30, 6, str(count), border=1, align='C')
        pdf.ln()

    # Treatment pie chart
    treatment_pie_chart = generate_pie_chart_with_arrows(clinician_name, treatment_types)

    # Ensure pie chart starts on a new page if the table height exceeds the page
    if pdf.get_y() + 80 > 270:  # Threshold for starting a new page
        pdf.add_page()

    pdf.ln(10)  # Add some space before the pie chart
    pdf.image(treatment_pie_chart, x=25, y=pdf.get_y() + 5, w=140, h=60)

    # Output the PDF file
    pdf_filename = f"{clinician_name.title()}_Report_{month_year.replace(' ', '_')}.pdf"
    pdf.output(pdf_filename)

    # Clean up temporary files
    if os.path.exists(treatment_pie_chart):
        os.remove(treatment_pie_chart)
    if ady_trend_graph and os.path.exists(ady_trend_graph):
        os.remove(ady_trend_graph)
    if chair_occupancy_trend_graph and os.path.exists(chair_occupancy_trend_graph):
        os.remove(chair_occupancy_trend_graph)
    if rebooking_trend_graph and os.path.exists(rebooking_trend_graph):
        os.remove(rebooking_trend_graph)

    return pdf_filename
# Generate reports for each clinician
pdf_files = []
month_year = "March 2025"
for index, row in final_df_with_patients.iterrows():
    clinician_name = row['User/Shop']
    turnover = row['Amount']
    ady = row['ADY']
    days_worked = row['Days Worked']
    unique_patients = row['Unique Patients Seen']
    chair_occupancy = row['Chair Occupancy (%)']
    rebooking_percentage = row['Rebooking Percentage (%)']
    future_plan_percentage = row['percentage_with_future_treatment_plans']

    # Treatment data for each clinician
    clinician_treatments = treatment_counts_df[treatment_counts_df['Clinician'] == clinician_name]
    treatment_types = clinician_treatments[['Treatment', 'Treatment Count']].values.tolist()

    # Generate the trend graphs
    ady_trend_graph = generate_line_graph(clinician_name, historical_data_df, "ADY")
    chair_occupancy_trend_graph = generate_line_graph(clinician_name, historical_data_df, "Chair Occupancy")
    rebooking_trend_graph = generate_line_graph(clinician_name, historical_data_df, "Rebooking Percentage")

    # Generate the PDF report for this clinician
    pdf_filename = generate_pdf(clinician_name, month_year, turnover, ady, group_avg_ady, days_worked,
        unique_patients, treatment_types, chair_occupancy, rebooking_percentage,
        future_plan_percentage, ady_trend_graph, chair_occupancy_trend_graph,
        rebooking_trend_graph)

    pdf_files.append(pdf_filename)

# Create a zip file containing all the PDF reports
zip_filename = 'clinician_reports_BB_March_2025.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for pdf_file in pdf_files:
        zipf.write(pdf_file)

# Download the zip file
files.download(zip_filename)

# Clean up PDF files after zipping
for pdf_file in pdf_files:
    if os.path.exists(pdf_file):
        os.remove(pdf_file)

print(f"Reports have been zipped into {zip_filename} and downloaded.")




Upload the cashbook file:


Saving CashbookwithIns_BB_2025-04-02_11-30.xlsx to CashbookwithIns_BB_2025-04-02_11-30.xlsx
Upload the average daily yield file:


Saving AverageDailyYie_BB_2025-04-02_11-31.xlsx to AverageDailyYie_BB_2025-04-02_11-31.xlsx
Upload the appointments file:


Saving AppointmentReport_BB_2025-04-02_11-21.xlsx to AppointmentReport_BB_2025-04-02_11-21.xlsx
Upload the chair occupancy file:


Saving chair occ bb.xlsx to chair occ bb.xlsx
Upload the rebooking rates file:


...

Saving Historical data BB.xlsx to Historical data BB.xlsx
Upload the future treatment plan analysis file:


Saving Clinician_Treatment_Plan_Summary_BB_3-25.xlsx to Clinician_Treatment_Plan_Summary_BB_3-25.xlsx
Enter Group Average ADY for the month: 1932


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Reports have been zipped into clinician_reports_BB_March_2025.zip and downloaded.
